In [0]:
!pip install cplex

     |████████████████████████████████| 31.0MB 141kB/s 


In [0]:
import cplex
import numpy as np

In [0]:
n = 30 * 3 * 5
boss_h = [6000000, 8000000, 15000000, 20000000, 20000000]
beat_boss_num = 3


y = np.random.normal(loc=350000, scale=100000, size=(30, 3, 5)).flatten()

In [0]:
ub = np.ones(n).tolist()
lb = np.zeros(n).tolist()
ctype = 'I' * n
colnames = np.array(['x{}'.format(i) for i in range(1, n + 1)]).reshape(30, 3, 5)

obj = -y

prob = cplex.Cplex()
prob.objective.set_sense(prob.objective.sense.minimize)
prob.variables.add(obj=obj,
                   lb=lb,
                   ub=ub,
                   types=ctype,
                   names=colnames.flatten().tolist())

# st.1
# 一刀只能被出在一个BOSS身上
rows = []
for i in range(30):
    for j in range(3):
        const = [colnames[i, j].tolist()]
        const.append(np.ones(5).tolist())
        rows.append(const)
prob.linear_constraints.add(lin_expr=rows,
                            rhs=np.ones(len(rows)).tolist(),
                            senses='E' * len(rows))

# st.2
# 每个人刀满3次
rows = []
for i in range(30):
    const = [colnames[i].flatten().tolist()]
    const.append(np.ones(len(const[0])).tolist())
    rows.append(const)
prob.linear_constraints.add(lin_expr=rows,
                            rhs=np.full(len(rows), 3).tolist(),
                            senses='E' * len(rows))

# st.3
# 至少刀死beat_boss_num个boss的血量
##const = [colnames.flatten().tolist()]
##const.append(y.tolist())
##rhs = [np.array(boss_h).dot(beat_boss)]
##prob.linear_constraints.add(lin_expr=[const],
##                            rhs=rhs,
##                            senses='G')

# st.4
# 不能过量刀BOSS, 并且至少刀死前beat_boss_num个boss
y = y.reshape(30, 3, 5)
rows = []
rhs = []
senses = ''
for i in range(beat_boss_num):
    const = [colnames[:, :, i].flatten().tolist()]
    const.append(y[:, :, i].flatten().tolist())
    rows.append(const)
    rhs.append(boss_h[i] + y.max())
    senses += 'L'
    
    rows.append(const)
    rhs.append(boss_h[i])
    senses += 'G'
    
prob.linear_constraints.add(lin_expr=rows,
                            rhs=rhs,
                            senses=senses)

range(120, 126)

In [0]:
prob.solve()
p = prob.solution.get_values()
p = np.array(p).reshape(30, 3, 5).astype(np.uint8)

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 90 columns.
MIP Presolve modified 360 coefficients.
Reduced MIP has 96 rows, 360 columns, and 900 nonzeros.
Reduced MIP has 360 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.88 ticks)
Found incumbent of value -3.1564408e+07 after 0.01 sec. (2.70 ticks)
Probing time = 0.00 sec. (0.16 ticks)
Tried aggregator 1 time.
Reduced MIP has 96 rows, 360 columns, and 900 nonzeros.
Reduced MIP has 360 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.51 ticks)
Probing time = 0.00 sec. (0.16 ticks)
Clique table members: 90.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 2 threads.
Root relaxation solution time = 0.00 sec. (0.25 ticks)

        Nodes                                         Cuts/
   Node 

In [0]:
# report
dmg = (p * y).sum()
print('\n\n造成的总伤害:', dmg)
for i in range(len(boss_h)):
    print(f'对{i+1}王造成伤害 [{round((p[:, :, i] * y[:, :, i]).sum(), 2)} / {boss_h[i]}]')

print('\n出刀表:\n', p)



造成的总伤害: 41754036.19890935
对1王造成伤害 [6473747.51 / 6000000]
对2王造成伤害 [8126332.51 / 8000000]
对3王造成伤害 [15063813.87 / 15000000]
对4王造成伤害 [5628993.81 / 20000000]
对5王造成伤害 [6461148.51 / 20000000]

出刀表:
 [[[0 0 0 1 0]
  [0 0 1 0 0]
  [0 0 1 0 0]]

 [[1 0 0 0 0]
  [0 0 0 0 1]
  [1 0 0 0 0]]

 [[0 1 0 0 0]
  [0 1 0 0 0]
  [0 0 1 0 0]]

 [[0 0 1 0 0]
  [0 0 1 0 0]
  [0 0 0 0 1]]

 [[0 0 1 0 0]
  [0 0 0 1 0]
  [0 0 1 0 0]]

 [[0 1 0 0 0]
  [0 1 0 0 0]
  [0 0 1 0 0]]

 [[0 0 1 0 0]
  [0 0 0 1 0]
  [1 0 0 0 0]]

 [[1 0 0 0 0]
  [0 1 0 0 0]
  [0 0 0 1 0]]

 [[0 0 1 0 0]
  [0 0 0 0 1]
  [1 0 0 0 0]]

 [[0 0 1 0 0]
  [0 0 0 1 0]
  [0 0 1 0 0]]

 [[0 0 0 1 0]
  [0 0 0 0 1]
  [0 0 1 0 0]]

 [[0 0 0 0 1]
  [0 0 1 0 0]
  [0 0 0 1 0]]

 [[0 0 1 0 0]
  [0 0 0 0 1]
  [1 0 0 0 0]]

 [[0 0 0 0 1]
  [0 1 0 0 0]
  [0 0 0 1 0]]

 [[0 1 0 0 0]
  [0 0 0 1 0]
  [0 1 0 0 0]]

 [[0 0 1 0 0]
  [0 0 0 1 0]
  [0 0 1 0 0]]

 [[1 0 0 0 0]
  [0 0 0 0 1]
  [0 1 0 0 0]]

 [[0 1 0 0 0]
  [0 0 1 0 0]
  [0 0 1 0 0]]

 [[0 0 1 0 0]
